In [245]:
import os

song_filepath = os.path.join('inputs', 'audios', 'Aventura - Enseñame A Olvidar.mp3')
song_metadata_filepath = os.path.join('inputs', 'audio_metadata', 'Aventura - Enseñame A Olvidar.txt')
moves_filepath = os.path.join('outputs', 'moves.json')

In [246]:
import json

# Open and read the JSON file
with open(moves_filepath, 'r') as file:
    moves_data = json.load(file)

musicality_map = {}
for move_id, move_data in moves_data.items():
    for index, item in enumerate(move_data):
        if item['musicality'] != '-':
            key = f"{item['count']}-{item['musicality']}"
            if key not in musicality_map:
                musicality_map[key] = []
            musicality_map[key].append({'move_id': move_id, 'index': index})
musicality_map

{'5-*': [{'move_id': '9da1eb8', 'index': 0},
  {'move_id': 'bfb0179', 'index': 13},
  {'move_id': '022dccf', 'index': 13},
  {'move_id': 'e3cbffd', 'index': 12},
  {'move_id': '4abbfb9', 'index': 7}],
 '7-*': [{'move_id': '9da1eb8', 'index': 2},
  {'move_id': 'acdbcc0', 'index': 15},
  {'move_id': 'acdbcc0', 'index': 23}],
 '5-*2': [{'move_id': '5d80c9a', 'index': 13}],
 '7-*2': [{'move_id': '5d80c9a', 'index': 15}],
 '1-*': [{'move_id': '8440066', 'index': 11},
  {'move_id': 'ea90f11', 'index': 8},
  {'move_id': 'ea90f11', 'index': 16},
  {'move_id': 'a0cfff7', 'index': 8},
  {'move_id': '022dccf', 'index': 9},
  {'move_id': '09e4474', 'index': 9}],
 '3-#2': [{'move_id': '8440066', 'index': 13}],
 '5-~2': [{'move_id': 'bfb0179', 'index': 5},
  {'move_id': 'acdbcc0', 'index': 13},
  {'move_id': '5365862', 'index': 5}],
 '7-~2': [{'move_id': 'bfb0179', 'index': 7}],
 '1-*3': [{'move_id': 'bfb0179', 'index': 9}],
 '7-#2': [{'move_id': 'bfb0179', 'index': 15},
  {'move_id': '1841f9d', 'in

In [247]:
with open(song_metadata_filepath, "r", encoding="utf-8") as f:
    musical_notation = f.read()
    # split() without arguments splits on any whitespace (spaces, tabs, newlines)
    musical_tokens = musical_notation.split()
num_tokens = len(musical_tokens)
num_tokens

72

In [248]:
from BeatNet.BeatNet import BeatNet
import os

os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'
estimator = BeatNet(1, mode='offline', inference_model='DBN', plot=[], thread=False)
beats = estimator.process(song_filepath)
beats

array([[ 0.32,  4.  ],
       [ 0.8 ,  1.  ],
       [ 1.28,  2.  ],
       [ 1.76,  3.  ],
       [ 2.24,  4.  ],
       [ 2.7 ,  1.  ],
       [ 3.18,  2.  ],
       [ 3.66,  3.  ],
       [ 4.16,  4.  ],
       [ 4.66,  1.  ],
       [ 5.14,  2.  ],
       [ 5.62,  3.  ],
       [ 6.1 ,  4.  ],
       [ 6.58,  1.  ],
       [ 7.04,  2.  ],
       [ 7.54,  3.  ],
       [ 8.  ,  4.  ],
       [ 8.48,  1.  ],
       [ 8.96,  2.  ],
       [ 9.44,  3.  ],
       [ 9.92,  4.  ],
       [10.38,  1.  ],
       [10.88,  2.  ],
       [11.38,  3.  ],
       [11.86,  4.  ],
       [12.34,  1.  ],
       [12.8 ,  2.  ],
       [13.3 ,  3.  ],
       [13.76,  4.  ],
       [14.24,  1.  ],
       [14.72,  2.  ],
       [15.2 ,  3.  ],
       [15.68,  4.  ],
       [16.16,  1.  ],
       [16.64,  2.  ],
       [17.14,  3.  ],
       [17.6 ,  4.  ],
       [18.08,  1.  ],
       [18.56,  2.  ],
       [19.04,  3.  ],
       [19.54,  4.  ],
       [20.02,  1.  ],
       [20.5 ,  2.  ],
       [20.

In [249]:
def get_important_beats(beats, num_tokens):
    # Find the first position where index == 1
    start = None
    for i, (time, index) in enumerate(beats):
        if index == 1:
            start = i
            break
    
    if start is None:
        raise ValueError("No row with index == 1 found")
    
    # Slice the subarray of length num_tokens (if enough rows exist)
    end = start + num_tokens
    if end > len(beats):
        raise ValueError("Not enough rows to get num_tokens elements starting at index==1")
    
    return beats[start:end]

important_beats = get_important_beats(beats, num_tokens)
important_beats

array([[ 0.8 ,  1.  ],
       [ 1.28,  2.  ],
       [ 1.76,  3.  ],
       [ 2.24,  4.  ],
       [ 2.7 ,  1.  ],
       [ 3.18,  2.  ],
       [ 3.66,  3.  ],
       [ 4.16,  4.  ],
       [ 4.66,  1.  ],
       [ 5.14,  2.  ],
       [ 5.62,  3.  ],
       [ 6.1 ,  4.  ],
       [ 6.58,  1.  ],
       [ 7.04,  2.  ],
       [ 7.54,  3.  ],
       [ 8.  ,  4.  ],
       [ 8.48,  1.  ],
       [ 8.96,  2.  ],
       [ 9.44,  3.  ],
       [ 9.92,  4.  ],
       [10.38,  1.  ],
       [10.88,  2.  ],
       [11.38,  3.  ],
       [11.86,  4.  ],
       [12.34,  1.  ],
       [12.8 ,  2.  ],
       [13.3 ,  3.  ],
       [13.76,  4.  ],
       [14.24,  1.  ],
       [14.72,  2.  ],
       [15.2 ,  3.  ],
       [15.68,  4.  ],
       [16.16,  1.  ],
       [16.64,  2.  ],
       [17.14,  3.  ],
       [17.6 ,  4.  ],
       [18.08,  1.  ],
       [18.56,  2.  ],
       [19.04,  3.  ],
       [19.54,  4.  ],
       [20.02,  1.  ],
       [20.5 ,  2.  ],
       [20.98,  3.  ],
       [21.

In [250]:
import numpy as np
beats_array = [(x % 8) + 1 for x in range(num_tokens)]
selected_beats = np.column_stack((important_beats, musical_tokens, beats_array, [f'{beats_array[index]}-{musical_tokens[index]}' for  index in range(num_tokens)])).tolist()
selected_beats

[['0.8', '1.0', '-', '1', '1--'],
 ['1.28', '2.0', '-', '2', '2--'],
 ['1.76', '3.0', '-', '3', '3--'],
 ['2.24', '4.0', '-', '4', '4--'],
 ['2.7', '1.0', '#4', '5', '5-#4'],
 ['3.18', '2.0', '-', '6', '6--'],
 ['3.66', '3.0', '-', '7', '7--'],
 ['4.16', '4.0', '-', '8', '8--'],
 ['4.66', '1.0', '-', '1', '1--'],
 ['5.14', '2.0', '-', '2', '2--'],
 ['5.62', '3.0', '-', '3', '3--'],
 ['6.1', '4.0', '-', '4', '4--'],
 ['6.58', '1.0', '-', '5', '5--'],
 ['7.04', '2.0', '-', '6', '6--'],
 ['7.54', '3.0', '-', '7', '7--'],
 ['8.0', '4.0', '-', '8', '8--'],
 ['8.48', '1.0', '~4', '1', '1-~4'],
 ['8.96', '2.0', '-', '2', '2--'],
 ['9.44', '3.0', '-', '3', '3--'],
 ['9.92', '4.0', '-', '4', '4--'],
 ['10.38', '1.0', '-', '5', '5--'],
 ['10.88', '2.0', '-', '6', '6--'],
 ['11.38', '3.0', '-', '7', '7--'],
 ['11.86', '4.0', '-', '8', '8--'],
 ['12.34', '1.0', '-', '1', '1--'],
 ['12.8', '2.0', '-', '2', '2--'],
 ['13.3', '3.0', '-', '3', '3--'],
 ['13.76', '4.0', '-', '4', '4--'],
 ['14.24', '1.

In [251]:
import random

musicality_points_map = {}
for index, (song_time, _, _, beat_index, musicality_token) in enumerate(selected_beats):
    if '--' not in musicality_token:
        musicality_points_map[index] = musicality_token

num_beats = len(selected_beats)
musicality_array = [False] * num_beats
for point, musicality_token in musicality_points_map.items():
    musicality_array[point] = True
    extension = musicality_token[-1]
    if extension.isdigit():
        for index in range(point, min(point + int(extension), num_beats)):
            musicality_array[index] = True
        
#musicality_points = sorted(musicality_points)

musicality_points_map = dict(sorted(musicality_points_map.items()))
musicality_points_map

{4: '5-#4', 16: '1-~4', 36: '5-#4', 68: '5-~4'}

In [252]:
musicality_array

[False,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 True,
 True]

In [253]:
combinatory_possibilities = []
for index, musicality_token in musicality_points_map.items():
    move_possibilities = []
    last_move_index = 0
    for move in musicality_map[musicality_token]:
        move_index = move['index']
        move_id = move['move_id']

        index_start_move = min(index - last_move_index, move_index)
        move_possibilities.append({
            'move_id': move_id,
            'song_index': index - index_start_move,
            'move_index': move_index - index_start_move,
            'song_end_index': index  + len(moves_data[move_id]) - move_index - 1
        })
    last_move_index = index
    extension = musicality_token[-1]
    if extension.isdigit():
        last_move_index = index + int(extension)
    combinatory_possibilities.append(move_possibilities)
combinatory_possibilities

[[{'move_id': 'ea90f11',
   'song_index': 0,
   'move_index': 0,
   'song_end_index': 16},
  {'move_id': '1841f9d',
   'song_index': 0,
   'move_index': 0,
   'song_end_index': 15},
  {'move_id': '7254257',
   'song_index': 0,
   'move_index': 1,
   'song_end_index': 15}],
 [{'move_id': '1841f9d',
   'song_index': 8,
   'move_index': 0,
   'song_end_index': 23},
  {'move_id': '5508f30',
   'song_index': 13,
   'move_index': 0,
   'song_end_index': 23},
  {'move_id': '4b8691c',
   'song_index': 7,
   'move_index': 0,
   'song_end_index': 23}],
 [{'move_id': 'ea90f11',
   'song_index': 32,
   'move_index': 0,
   'song_end_index': 48},
  {'move_id': '1841f9d',
   'song_index': 32,
   'move_index': 0,
   'song_end_index': 47},
  {'move_id': '7254257',
   'song_index': 31,
   'move_index': 0,
   'song_end_index': 47}],
 [{'move_id': '3e7415f',
   'song_index': 53,
   'move_index': 0,
   'song_end_index': 72},
  {'move_id': '5365862',
   'song_index': 55,
   'move_index': 0,
   'song_end_ind

In [254]:
from itertools import product
import copy

choreography_musicality_candidates = [
    [copy.deepcopy(d) for d in p]
    for p in product(*combinatory_possibilities)
    if len({d['move_id'] for d in p}) == len(p)
]
choreography = choreography_musicality_candidates[6]
choreography = sorted(choreography, key=lambda x: x['song_index'])
choreography

[{'move_id': 'ea90f11',
  'song_index': 0,
  'move_index': 0,
  'song_end_index': 16},
 {'move_id': '1841f9d',
  'song_index': 8,
  'move_index': 0,
  'song_end_index': 23},
 {'move_id': '7254257',
  'song_index': 31,
  'move_index': 0,
  'song_end_index': 47},
 {'move_id': 'cc02778',
  'song_index': 52,
  'move_index': 0,
  'song_end_index': 79}]

In [255]:
def get_initial_extended_choreo(choreography, moves_data,  num_beats):
    extended_choreo = [False] * num_beats
    last_move = None
    similarity_scores = []
    exclude_moves = []
    
    for move in choreography:
        if last_move and last_move['song_end_index'] >= move['song_index']:
            best_similarity = 0.0
            best_index = move['song_index']
            for index in range(move['song_index'], last_move['song_end_index'] + 1):
                last_move_id = last_move['move_id']
                move_id = move['move_id']
                last_move_index = last_move['move_index'] + index - last_move['song_index']
                move_index = move['move_index'] + index - move['song_index']
    
                for connection in moves_data[move_id][move_index]['connections']:
                    if connection['move_id'] == last_move_id and connection['index'] == last_move_index:
                        if connection['similarity'] > best_similarity:
                            best_similarity = connection['similarity']
                            best_index = index
                            break
    
            similarity_scores.append((best_index, best_similarity))
            for index in range(move['song_index'], best_index + 1):
                extended_choreo[index] = {
                    'move_id': last_move['move_id'],
                    'move_index': last_move['move_index'] + index - last_move['song_index'],
                    'song_index': index
                }
    
            for index in range(best_index, last_move['song_end_index'] + 1):
                extended_choreo[index] = {
                    'move_id': move['move_id'],
                    'move_index': move['move_index'] + index - move['song_index'],
                    'song_index': index
                }
    
            move['song_index'] = index
    
        for index in range(move['song_index'], min(move['song_end_index'], num_beats)):
            if not extended_choreo[index]: 
                extended_choreo[index] = {
                    'move_id': move['move_id'],
                    'move_index': move['move_index'] + index - move['song_index'],
                    'song_index': index
                }
            else:
                pass
        last_move = move
    return extended_choreo, similarity_scores

def find_false_ranges(arr):
    ranges = []
    start = None

    for i, val in enumerate(arr):
        if val is False:                     # start of a False block
            if start is None:
                start = i
        else:                                # end of a False block
            if start is not None:
                ranges.append((start, i - 1))
                start = None

    # If the list ends with False block
    if start is not None:
        ranges.append((start, len(arr) - 1))

    return ranges

def refine_choreo(extended_choreo, blank_ranges, moves_data, num_beats, similarity_scores):
    for blank_range in blank_ranges:
        blank_size = blank_range[1] + 1 - blank_range[0]
        # For now, I'll try to chain only one move maximizing the similarity
        last_move = None
        if blank_range[0] > 0:
            last_move = extended_choreo[blank_range[0] - 1]
        next_move = None
        if blank_range[1] + 1 < num_beats:
            next_move = extended_choreo[blank_range[1] + 1]
    
        best_cummulative_similarity = -1.0
        best_index = 0
        best_move_id = ''
        if last_move is not None and next_move is not None:
            for start_connection in moves_data[last_move['move_id']][last_move['move_index']]['connections']:
                for finish_connection in moves_data[next_move['move_id']][next_move['move_index']]['connections']:
                    if finish_connection['move_id'] == start_connection['move_id'] and finish_connection['index'] == start_connection['index'] + blank_size + 1:
                        local_similarity = start_connection['similarity'] + finish_connection['similarity']
                        if local_similarity > best_cummulative_similarity:
                            best_cummulative_similarity = local_similarity
                            best_index = start_connection['index']
                            best_move_id = start_connection['move_id']
                        break
    
            similarity_scores.append((last_move['song_index'] + 1, start_connection['similarity']))
            similarity_scores.append((next_move['song_index'], finish_connection['similarity']))
        elif last_move is not None:
            for connection in moves_data[last_move['move_id']][last_move['move_index']]['connections']:
                if len(moves_data[connection['move_id']][connection['index']:]) >= blank_size:
                    if connection['similarity'] > best_cummulative_similarity:
                        best_cummulative_similarity = connection['similarity']
                        best_index = connection['index']
                        best_move_id = connection['move_id']
                    break
            similarity_scores.append((last_move['song_index'] + 1, connection['similarity']))
        elif next_move is not None:
            for connection in moves_data[next_move['move_id']][next_move['move_index']]['connections']:
                if connection['index'] - blank_size >= 0:
                    if connection['similarity'] > best_cummulative_similarity:
                        best_cummulative_similarity = connection['similarity']
                        best_index = connection['index'] - blank_size
                        best_move_id = connection['move_id']
                    break
            similarity_scores.append((next_move['song_index'], connection['similarity']))
    
        for index in range(blank_range[0], blank_range[1] + 1):
            extended_choreo[index] = {
                'move_id': best_move_id,
                'move_index': best_index + index - blank_range[0],
                'song_index': index
            }
    return extended_choreo, similarity_scores


def refine_choreo2(
    extended_choreo, blank_ranges, moves_data, num_beats,
    similarity_scores, musicality_array
):
    for blank_range in blank_ranges:
        start = blank_range[0]
        end = blank_range[1]
        blank_size = end - start + 1

        # --------------------------
        # SEARCH LAST MOVE BACKWARD
        # --------------------------
        last_candidates = []
        idx = start - 1
        while idx >= 0:
            last_candidates.append(extended_choreo[idx])
            if musicality_array[idx]:
                break
            idx -= 1

        # --------------------------
        # SEARCH NEXT MOVE FORWARD
        # --------------------------
        next_candidates = []
        idx = end + 1
        while idx < num_beats:
            next_candidates.append(extended_choreo[idx])
            if musicality_array[idx]:
                break
            idx += 1

        best_similarity = -1.0
        next_similarity = -1.0
        last_similarity = -1.0
        best_move_id = "-"
        best_index = 0
        s_start_index = start
        s_end_index = end

        # ----------------------------------------------------------
        # CASE 1: We have backwards moves AND forward moves
        # ----------------------------------------------------------
        if last_candidates and next_candidates:
            for last_move in last_candidates:
                lm_id = last_move["move_id"]
                lm_idx = last_move["move_index"]
                for next_move in next_candidates:
                    nm_id = next_move["move_id"]
                    nm_idx = next_move["move_index"]

                    for start_conn in moves_data[lm_id][lm_idx]["connections"]:
                        for finish_conn in moves_data[nm_id][nm_idx]["connections"]:
                            if (
                                finish_conn["move_id"] == start_conn["move_id"]
                                and finish_conn["index"] == start_conn["index"] + next_move['song_index'] - last_move['song_index']
                                and finish_conn["move_id"] != nm_id
                                and start_conn["move_id"] != lm_id
                            ):
                                local_sim = start_conn["similarity"] + finish_conn["similarity"]
                                if local_sim > best_similarity:
                                    s_start_index = last_move['song_index'] + 1
                                    s_end_index = next_move['song_index']
                                    next_similarity = finish_conn["similarity"]
                                    last_similarity = start_conn["similarity"]
                                    best_similarity = local_sim
                                    best_move_id = start_conn["move_id"]
                                    best_index = start_conn["index"]

            similarity_scores.append((s_start_index, last_similarity))
            similarity_scores.append((s_end_index, next_similarity))

        # ----------------------------------------------------------
        # CASE 2: Only last candidates (no next moves)
        # ----------------------------------------------------------
        elif last_candidates:
            for last_move in last_candidates:
                lm_id = last_move["move_id"]
                lm_idx = last_move["move_index"]

                for conn in moves_data[lm_id][lm_idx]["connections"]:
                    # we need enough indices ahead for the blank
                    if len(moves_data[conn["move_id"]]) >= conn["index"] + num_beats - last_move['song_index'] - 1:
                        if conn["similarity"] > best_similarity:
                            s_start_index = last_move['song_index'] + 1
                            best_similarity = conn["similarity"]
                            best_index = conn["index"]
                            best_move_id = conn["move_id"]
            similarity_scores.append((s_start_index, best_similarity))

        # ----------------------------------------------------------
        # CASE 3: Only next candidates (no last moves)
        # ----------------------------------------------------------
        elif next_candidates:
            for next_move in next_candidates:
                nm_id = next_move["move_id"]
                nm_idx = next_move["move_index"]

                for conn in moves_data[nm_id][nm_idx]["connections"]:
                    start_index = conn["index"] - next_move['song_index']
                    if start_index >= 0:
                        if conn["similarity"] > best_similarity:
                            s_end_index = next_move['song_index']
                            best_similarity = conn["similarity"]
                            best_index = start_index
                            best_move_id = conn["move_id"]
            similarity_scores.append((s_end_index, best_similarity))

        # ----------------------------------------------------------
        # FILL THE BLANK WITH THE SELECTED MOVE
        # ----------------------------------------------------------
        for pos in range(s_start_index, s_end_index + 1):
            extended_choreo[pos] = {
                "move_id": best_move_id,
                "move_index": best_index + (pos - s_start_index),
                "song_index": pos,
            }

    return extended_choreo, similarity_scores

def simplify_choreo(extended_choreo):
    choreography = []
    current = None

    for item in extended_choreo:
        if isinstance(item, dict):
            if current is None:
                # Start a new block
                current = {
                    "move_id": item["move_id"],
                    "song_index": item["song_index"],
                    "move_index": item["move_index"],
                    "song_end_index": item["song_index"] + 1
                }
            else:
                # Continue same block
                if item["move_id"] == current["move_id"]:
                    current["song_end_index"] = item["song_index"] + 1
                else:
                    # Block ended, push & start a new one
                    choreography.append(current)
                    current = {
                        "move_id": item["move_id"],
                        "song_index": item["song_index"],
                        "move_index": item["move_index"],
                        "song_end_index": item["song_index"] + 1
                    }
        else:
            # False encountered: close current block if open
            if current is not None:
                choreography.append(current)
                current = None

    # Catch last block
    if current is not None:
        choreography.append(current)

    return choreography

def simplify_choreo2(extended_choreo):
    choreography = []
    current = None

    for item in extended_choreo:
        if not isinstance(item, dict):
            # Close block on False
            if current is not None:
                choreography.append(current)
                current = None
            continue

        if current is None:
            # Start new block
            current = {
                "move_id": item["move_id"],
                "song_index": item["song_index"],
                "move_index": item["move_index"],  # <-- store starting index
                "song_end_index": item["song_index"] + 1
            }
        else:
            # Same move_id AND strictly sequential move_index?
            same_move = (item["move_id"] == current["move_id"])
            sequential = (item["move_index"] == current["move_index"] + (current["song_end_index"] - current["song_index"]))

            # The old bug: DO NOT update move_index ever again
            if same_move and sequential:
                current["song_end_index"] = item["song_index"] + 1
            else:
                # End block, start new one
                choreography.append(current)
                current = {
                    "move_id": item["move_id"],
                    "song_index": item["song_index"],
                    "move_index": item["move_index"],  # <-- correct
                    "song_end_index": item["song_index"] + 1
                }

    if current is not None:
        choreography.append(current)

    return choreography


extended_choreo, similarity_scores = get_initial_extended_choreo(choreography, moves_data,  num_beats)
blank_ranges = find_false_ranges(extended_choreo)
extended_choreo2, similarity_scores = refine_choreo2(extended_choreo.copy(), blank_ranges, moves_data, num_beats, similarity_scores, musicality_array)
choreography = simplify_choreo2(extended_choreo2)
choreography

[{'move_id': 'ea90f11',
  'song_index': 0,
  'move_index': 0,
  'song_end_index': 12},
 {'move_id': '1841f9d',
  'song_index': 12,
  'move_index': 4,
  'song_end_index': 17},
 {'move_id': '1841f9d',
  'song_index': 17,
  'move_index': 1,
  'song_end_index': 23},
 {'move_id': '9da1eb8',
  'song_index': 23,
  'move_index': 2,
  'song_end_index': 34},
 {'move_id': '7254257',
  'song_index': 34,
  'move_index': 3,
  'song_end_index': 47},
 {'move_id': 'bfb0179',
  'song_index': 47,
  'move_index': 7,
  'song_end_index': 54},
 {'move_id': 'cc02778',
  'song_index': 54,
  'move_index': 2,
  'song_end_index': 72}]

In [256]:
choreography_candidates = []
for index, choreography_candidate in enumerate(choreography_musicality_candidates):
    choreography_candidate = sorted([copy.deepcopy(d) for d in choreography_candidate], key=lambda x: x['song_index'])
    extended_choreo, similarity_scores = get_initial_extended_choreo(choreography_candidate, moves_data,  num_beats)
    blank_ranges = find_false_ranges(extended_choreo)
    # TODO: Expand the capabilities of refine_choreo to extend to try many moves
    extended_choreo, similarity_scores = refine_choreo2(extended_choreo, blank_ranges, moves_data, num_beats, similarity_scores, musicality_array)
    similarity_scores = [x for x in similarity_scores if x[1] < 1.0]

    choreography = simplify_choreo2(extended_choreo)
    is_valid_choreo = True
    for move in choreography:
        if move['move_id'] == '':
            is_valid_choreo = False
            break
    if choreography[-1]['song_end_index'] != num_beats:
        is_valid_choreo = False
    if is_valid_choreo:
        choreography_candidates.append((choreography, similarity_scores, index))
choreography_candidates = sorted(choreography_candidates, key=lambda x: sum([item[1] for item in x[1]]) / len(x[1]), reverse=True)

In [257]:
choreography_candidates[0]

([{'move_id': 'ea90f11',
   'song_index': 0,
   'move_index': 0,
   'song_end_index': 12},
  {'move_id': '1841f9d',
   'song_index': 12,
   'move_index': 4,
   'song_end_index': 17},
  {'move_id': '1841f9d',
   'song_index': 17,
   'move_index': 1,
   'song_end_index': 23},
  {'move_id': '9da1eb8',
   'song_index': 23,
   'move_index': 2,
   'song_end_index': 34},
  {'move_id': '7254257',
   'song_index': 34,
   'move_index': 3,
   'song_end_index': 47},
  {'move_id': 'acdbcc0',
   'song_index': 47,
   'move_index': 7,
   'song_end_index': 58},
  {'move_id': '5365862',
   'song_index': 58,
   'move_index': 3,
   'song_end_index': 72}],
 [(12, 0.9667478929013231),
  (23, 0.9904377687942276),
  (33, 0.9547250235766838),
  (47, 0.9482251546420287),
  (57, 0.965752453581926)],
 1)

In [ ]:
from moviepy import VideoFileClip, AudioFileClip, ColorClip, concatenate_videoclips, vfx
import numpy as np
import os

music_beats = [float(b[0]) for b in selected_beats]

# Song audio
song = AudioFileClip(song_filepath)
song_duration = song.duration


# --- Helper: fit move video to beats ---
def stretch_move(move_id, move_index, song_start_idx, song_end_idx):
    # Number of beats in song slice
    n_song_beats = song_end_idx - song_start_idx + 1

    # Take only that many beats from the move, starting at move_index
    move_beats = [d['time'] for d in moves_data[move_id][move_index:move_index+n_song_beats]]

    if len(move_beats) < 2:
        return None  # not enough beats in the move

    move_duration = move_beats[-1] - move_beats[0]

    # Adjust the end of the move
    song_end_idx = min(song_start_idx + len(move_beats) - 1, song_end_idx)

    # Target beats slice in the song
    target_beats = music_beats[song_start_idx:song_end_idx + 1]
    target_duration = target_beats[-1] - target_beats[0]

    # Load the clip
    clip = VideoFileClip(os.path.join('inputs', 'videos', f"{move_id}.mp4"))

    # Trim to the move duration (relative to first beat)
    clip = clip.subclipped(move_beats[0], move_beats[0] + move_duration)

    # Stretch to fit target beats duration
    clip = clip.with_effects([vfx.MultiplySpeed(factor=move_duration/target_duration)])

    # Force exact duration
    clip = clip.with_duration(target_duration)

    return clip


# --- Build timeline ---
for idx, choreography_data in enumerate(choreography_candidates[:3]):
    choreography = [copy.deepcopy(d) for d in choreography_data[0]]
    clips = []
    current_time = 0
    
    for i, move in enumerate(choreography):
        song_idx = move['song_index']
        next_song_idx = choreography[i+1]['song_index'] if i+1 < len(choreography) else len(music_beats)
    
        # black screen before this move (if gap)
        beat_time = music_beats[song_idx]
        if beat_time > current_time:
            clips.append(ColorClip(size=(640,480), color=(0,0,0), duration=beat_time-current_time))

        # add stretched move
        move_clip = stretch_move(
            move['move_id'],
            move['move_index'],
            song_idx,
            next_song_idx
        )
        clips.append(move_clip)
    
        current_time = beat_time + move_clip.duration
    
    # black screen until song ends
    if current_time < song_duration:
        clips.append(ColorClip(size=(640,480), color=(0,0,0), duration=song_duration-current_time))
    
    # Concatenate all
    final_video = concatenate_videoclips(clips, method="compose")
    
    # Set audio
    final_video.audio = song
    
    # Export
    choreo_filepath = os.path.join('outputs', f'Aventura - Enseñame A Olvidar - Version {idx + 1}.mp4')
    final_video.write_videofile(choreo_filepath, codec="libx264", audio_codec="aac")

In [ ]:
choreography_candidates[0]

In [ ]:
len(moves_data['cc02778'])